In [1]:
import pandas as pd
import seaborn as sns
import datetime as dt

# Import the data

c_data = \
    pd.read_csv("jakarta-central (us consulate), indonesia-air-quality.csv", 
    header = 0, 
    names = ["Date", "PM25", "PM10"])
c_data["Loc"] = "Central Jakarta"

s_data = \
    pd.read_csv("jakarta-south (us consulate), indonesia-air-quality.csv", 
    header = 0, 
    names = ["Date", "PM25", "PM10"])
s_data["Loc"] = "South Jakarta"

# Combine the data for Central Jakarta and South Jakarta

data = pd.concat([c_data, s_data])

In [2]:
data.head()

,Date,PM25,PM10,Loc
0,2022/10/1,86,,Central Jakarta
1,2022/10/2,47,,Central Jakarta
2,2022/10/3,84,,Central Jakarta
3,2022/10/4,66,,Central Jakarta
4,2022/10/5,71,,Central Jakarta


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4959 entries, 0 to 2461
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    4959 non-null   object
 1   PM25    4959 non-null   object
 2   PM10    4959 non-null   object
 3   Loc     4959 non-null   object
dtypes: object(4)
memory usage: 193.7+ KB


In [4]:
# Calculate the AQI as the maximum values between PM25 and PM10

data["AQI"] = data[["PM25", "PM10"]].max(axis = 1)

# Drop PM25 and PM10 columns

data.drop(["PM25", "PM10"], axis = 1, inplace = True)

# Convert date column to datetime format

data["Date"] = pd.to_datetime(data["Date"])

In [5]:
data.head()

,Date,Loc,AQI
0,2022-10-01,Central Jakarta,86.0
1,2022-10-02,Central Jakarta,47.0
2,2022-10-03,Central Jakarta,84.0
3,2022-10-04,Central Jakarta,66.0
4,2022-10-05,Central Jakarta,71.0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4959 entries, 0 to 2461
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    4959 non-null   datetime64[ns]
 1   Loc     4959 non-null   object        
 2   AQI     4959 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 155.0+ KB


In [10]:
# Drop data below 2016 (the data is incomplete)

data["Year"] = data["Date"].dt.year
data["Month"] = data["Date"].dt.strftime("%Y-%m")
data.drop(index = data[data["Year"] < 2016].index, inplace = True)

In [11]:
# Group data (monthly)

data_monthly = \
    data.drop(columns = ["Date", "Year"]).\
    groupby(["Month", "Loc"]).\
    agg(["min", "mean", "max"]).\
    reset_index()
data_monthly.columns = \
    [" ".join(column) for column in data_monthly.columns.to_flat_index()]
data_monthly.columns = \
    [column.strip() for column in data_monthly.columns]

In [12]:
data_monthly

,Month,Loc,AQI min,AQI mean,AQI max
0,2016-01,Central Jakarta,43.0,78.586207,121.0
1,2016-01,South Jakarta,50.0,90.655172,138.0
2,2016-02,Central Jakarta,45.0,76.304348,120.0
3,2016-02,South Jakarta,36.0,86.068966,147.0
4,2016-03,Central Jakarta,53.0,93.129032,138.0
...,...,...,...,...,...
159,2022-08,South Jakarta,88.0,110.291667,150.0
160,2022-09,Central Jakarta,53.0,93.200000,131.0
161,2022-09,South Jakarta,68.0,111.800000,149.0
162,2022-10,Central Jakarta,47.0,75.636364,119.0


In [13]:
# Group date (yearly)

data_yearly = \
    data.drop(columns = ["Date", "Month"]).\
    groupby(["Year", "Loc"]).\
    agg(["min", "mean", "max"]).\
    reset_index()
data_yearly.columns = \
    [" ".join(column) for column in data_yearly.columns.to_flat_index()]
data_yearly.columns = \
    [column.strip() for column in data_yearly.columns]

In [14]:
data_yearly

,Year,Loc,AQI min,AQI mean,AQI max
0,2016,Central Jakarta,32.0,99.891061,209.0
1,2016,South Jakarta,33.0,101.972527,165.0
2,2017,Central Jakarta,7.0,79.558282,148.0
3,2017,South Jakarta,7.0,82.298780,165.0
4,2018,Central Jakarta,12.0,73.144509,161.0
5,2018,South Jakarta,4.0,77.495702,209.0
6,2019,Central Jakarta,9.0,77.811798,159.0
7,2019,South Jakarta,23.0,83.713873,165.0
8,2020,Central Jakarta,7.0,62.746479,137.0
9,2020,South Jakarta,6.0,56.741758,140.0
